In [1]:
import keras
from keras import regularizers
import numpy as np
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import RMSprop
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential, Input
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D
from keras.layers import LSTM, Bidirectional, Activation
from keras.callbacks import ModelCheckpoint
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt 
%matplotlib inline

Using TensorFlow backend.


In [2]:
#hyperparamter
DATA_DIR = "../data/"
output_dir = '../data/lstm/'

#data
test_size = 0.1
random_state = 12345

#Training 
epochs = 4
batch_size = 128

#vector-space embedding
n_dim = 64
n_unique_words = 5000
max_review_length = 100
pad_type = trunc_type = 'pre'
drop_embed = 0.25

#lstm architecture 
n_lstm = 256
drop_lstm = 0.2

#optimizer
rms

In [3]:
# load the data
data, labels = [],[]
for file in tqdm(os.listdir(DATA_DIR+'pickled/')):
    with open(DATA_DIR+'pickled/'+file,'rb') as f:
        text,label = pickle.load(f)
        data.append(text)
        labels.append(label)

  2%|▏         | 1158/51915 [00:09<07:17, 116.13it/s]/home/bt0/16CS10053/anaconda3/envs/cgcnn/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 51915/51915 [02:19<00:00, 371.38it/s]


In [4]:
len(data)

51915

In [5]:
one_hot = MultiLabelBinarizer()
labels = one_hot.fit_transform(labels)
len(labels[0])

62

In [6]:
data = np.array(data)
labels = np.array(labels)


In [7]:
train_x,test_x, train_y, test_y = train_test_split(data, labels, test_size=test_size, random_state=random_state)

In [8]:
len(train_x), len(train_y), len(test_x), len(test_y)

(46723, 46723, 5192, 5192)

In [9]:
train_y = [x for x in train_y.transpose()]
test_y = [x for x in test_y.transpose()]

In [10]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [11]:
MAX_WORDS=25000
train_with_cat=[]
MAX_LEN=100

tok_1=Tokenizer(num_words=MAX_WORDS)
tok_1.fit_on_texts(train_x)                                                       #converting text input to vector
sequences = tok_1.texts_to_sequences(train_x)                                     #to convert sentence to sequences of numbers
sequences_matrix = sequence.pad_sequences(sequences, maxlen=MAX_LEN)
def RNN_1():
    inputs = Input(name='inputs', shape=[MAX_LEN])
    layer = Embedding(MAX_WORDS, 64, input_length=MAX_LEN)(inputs)  
    layer = Bidirectional(LSTM(64,recurrent_dropout=0.1,kernel_regularizer=regularizers.l2(0.000014),activity_regularizer=regularizers.l1(0.00012)))(layer)
    layer_lst=[]
    sigmoid_lst=[]
    for i in range(len(train_y)):
        layer_temp=Dense(64,name='FC_1'+str(i),kernel_regularizer=regularizers.l2(0.000013))(layer)
        temp=Dense(1,name='FC_2'+str(i),kernel_regularizer=regularizers.l2(0.000013))(layer_temp)
        sigmoid_lst.append(Activation('sigmoid')(temp))

    model = Model(inputs=inputs,outputs=sigmoid_lst)
    return model
model = RNN_1()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 64)      1600000     inputs[0][0]                     
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 128)          66048       embedding_1[0][0]                
__________________________________________________________________________________________________
FC_10 (Dense)        

In [13]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.0007),metrics=['accuracy'])
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5", save_best_only=True)
model.fit(sequences_matrix,
          train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          validation_split=0.1, 
          callbacks=[modelcheckpoint])
model.save('LSTM2_main.h5')

Train on 42050 samples, validate on 4673 samples
Epoch 1/20
42050/42050 [==============================] - 74s 2ms/step - loss: 4.3486 - activation_1_loss: 0.0028 - activation_2_loss: 0.0793 - activation_3_loss: 0.0686 - activation_4_loss: 0.0267 - activation_5_loss: 0.0033 - activation_6_loss: 0.0024 - activation_7_loss: 0.0591 - activation_8_loss: 0.1028 - activation_9_loss: 0.0039 - activation_10_loss: 0.0768 - activation_11_loss: 0.2309 - activation_12_loss: 0.0024 - activation_13_loss: 0.3300 - activation_14_loss: 0.0505 - activation_15_loss: 0.0752 - activation_16_loss: 0.1469 - activation_17_loss: 0.1786 - activation_18_loss: 0.0784 - activation_19_loss: 0.0759 - activation_20_loss: 2.9083e-04 - activation_21_loss: 0.0553 - activation_22_loss: 0.0465 - activation_23_loss: 0.0330 - activation_24_loss: 0.0382 - activation_25_loss: 0.0062 - activation_26_loss: 0.0736 - activation_27_loss: 0.0900 - activation_28_loss: 0.0110 - activation_29_loss: 0.0093 - activation_30_loss: 0.0440 

  256/42050 [..............................] - ETA: 55s - loss: 4.0104 - activation_1_loss: 7.0312e-05 - activation_2_loss: 0.0593 - activation_3_loss: 0.0681 - activation_4_loss: 0.0402 - activation_5_loss: 1.8244e-04 - activation_6_loss: 1.0752e-04 - activation_7_loss: 0.0206 - activation_8_loss: 0.1403 - activation_9_loss: 2.8101e-04 - activation_10_loss: 0.0522 - activation_11_loss: 0.1681 - activation_12_loss: 8.9676e-05 - activation_13_loss: 0.3777 - activation_14_loss: 0.0967 - activation_15_loss: 0.0956 - activation_16_loss: 0.1200 - activation_17_loss: 0.1947 - activation_18_loss: 0.0762 - activation_19_loss: 0.0705 - activation_20_loss: 7.0583e-06 - activation_21_loss: 0.0246 - activation_22_loss: 0.0216 - activation_23_loss: 0.0058 - activation_24_loss: 0.0351 - activation_25_loss: 9.9328e-04 - activation_26_loss: 0.0646 - activation_27_loss: 0.0565 - activation_28_loss: 0.0438 - activation_29_loss: 0.0013 - activation_30_loss: 0.0097 - activation_31_loss: 0.0024 - activatio

  256/42050 [..............................] - ETA: 55s - loss: 4.2126 - activation_1_loss: 1.7534e-04 - activation_2_loss: 0.0815 - activation_3_loss: 0.1281 - activation_4_loss: 0.0185 - activation_5_loss: 2.6544e-04 - activation_6_loss: 7.6400e-05 - activation_7_loss: 0.0300 - activation_8_loss: 0.1014 - activation_9_loss: 2.2431e-04 - activation_10_loss: 0.0605 - activation_11_loss: 0.2211 - activation_12_loss: 1.4906e-04 - activation_13_loss: 0.3385 - activation_14_loss: 0.0442 - activation_15_loss: 0.0977 - activation_16_loss: 0.1969 - activation_17_loss: 0.1104 - activation_18_loss: 0.0795 - activation_19_loss: 0.0552 - activation_20_loss: 4.9938e-06 - activation_21_loss: 0.0859 - activation_22_loss: 0.0423 - activation_23_loss: 0.0605 - activation_24_loss: 0.0717 - activation_25_loss: 7.1641e-04 - activation_26_loss: 0.0602 - activation_27_loss: 0.0783 - activation_28_loss: 0.0014 - activation_29_loss: 0.0015 - activation_30_loss: 0.0685 - activation_31_loss: 0.0190 - activatio

38016/42050 [==========================>...] - ETA: 5s - loss: 3.9761 - activation_1_loss: 0.0027 - activation_2_loss: 0.0776 - activation_3_loss: 0.0638 - activation_4_loss: 0.0251 - activation_5_loss: 0.0027 - activation_6_loss: 0.0024 - activation_7_loss: 0.0432 - activation_8_loss: 0.0948 - activation_9_loss: 0.0037 - activation_10_loss: 0.0625 - activation_11_loss: 0.2146 - activation_12_loss: 0.0023 - activation_13_loss: 0.3122 - activation_14_loss: 0.0443 - activation_15_loss: 0.0696 - activation_16_loss: 0.1375 - activation_17_loss: 0.1542 - activation_18_loss: 0.0723 - activation_19_loss: 0.0718 - activation_20_loss: 3.3125e-04 - activation_21_loss: 0.0439 - activation_22_loss: 0.0340 - activation_23_loss: 0.0310 - activation_24_loss: 0.0354 - activation_25_loss: 0.0057 - activation_26_loss: 0.0661 - activation_27_loss: 0.0767 - activation_28_loss: 0.0094 - activation_29_loss: 0.0087 - activation_30_loss: 0.0405 - activation_31_loss: 0.0175 - activation_32_loss: 0.0803 - activ

KeyboardInterrupt: 